In [5]:
import copy
import pylab
import numpy as np
from envorig import Env
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential
from keras import backend as K
%run envorig.py

EPISODES = 2500

# this is REINFORCE Agent for GridWorld
class ReinforceAgent:
    def __init__(self):
        self.load_model = False
        # actions which agent can do
        self.action_space = [0, 1, 2, 3, 4]
        # get size of state and action
        self.action_size = len(self.action_space)
        self.state_size = 15
        self.discount_factor = 0.99
        self.learning_rate = 0.001

        self.model = self.build_model()
        self.optimizer = self.optimizer()
        self.states, self.actions, self.rewards = [], [], []

        if self.load_model:
            self.model.load_weights('./save_model/2Agent10times10_1.h5')

    # state is input and probability of each action(policy) is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='softmax'))
        model.summary()
        return model

    # create error function and training function to update policy network
    def optimizer(self):
        action = K.placeholder(shape=[None, 5])
        discounted_rewards = K.placeholder(shape=[None, ])

        # Calculate cross entropy error function
        action_prob = K.sum(action * self.model.output, axis=1)
        cross_entropy = K.log(action_prob) * discounted_rewards
        loss = -K.sum(cross_entropy)

        # create training function
        optimizer = Adam(lr=self.learning_rate)
        updates = optimizer.get_updates(self.model.trainable_weights, [],
                                        loss)
        train = K.function([self.model.input, action, discounted_rewards], [],
                           updates=updates)

        return train

    # get action from policy network, see np.random.choice instructions
    def get_action(self, state):
        policy = self.model.predict(state)[0]
        return np.random.choice(self.action_size, 1, p=policy)[0] 

    # calculate discounted rewards
    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, len(rewards))):
            running_add = running_add * self.discount_factor + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    # save states, actions and rewards for an episode
    def append_sample(self, state, action, reward):
        self.states.append(state[0])
        self.rewards.append(reward)
        act = np.zeros(self.action_size)
        act[action] = 1
        self.actions.append(act)

    # update policy neural network
    def train_model(self):
        discounted_rewards = np.float32(self.discount_rewards(self.rewards))
        discounted_rewards -= np.mean(discounted_rewards)
        discounted_rewards /= np.std(discounted_rewards)

        self.optimizer([self.states, self.actions, discounted_rewards])
        self.states, self.actions, self.rewards = [], [], []


if __name__ == "__main__":
    env = Env()
    agent = ReinforceAgent()

    global_step = 0
    scores, episodes = [], []

    for e in range(EPISODES):
        done = False
        score = 0
        # fresh env
        state = env.reset()
        #state.extend((0,0,0))
        state = np.reshape(state, [1, 15])

        while not done:
            #start = time.time()
            global_step += 1
            # get action for the current state and go one step in environment
            action = agent.get_action(state)
            next_state, reward, done = env.step(action)
            #next_state.extend((0,0,0))
            next_state = np.reshape(next_state, [1, 15])

            agent.append_sample(state, action, reward)
            score += reward
            state = copy.deepcopy(next_state)

            #end = time.time()
            #print(end-start)
            
            if done:
                # update policy neural network for each episode
                agent.train_model()
                scores.append(score)
                episodes.append(e)
                score = round(score, 2)
                if e % 100 == 0:
                    print("episode:", e, "  score:", score, "  time_step:",
                          global_step)
        if e == 2499:
            agent
        if e % 100 == 0:
            #pylab.plot(episodes, scores, 'b')
            #pylab.savefig("./save_graph/1Agent10times10.eps")
            agent.model.save_weights("./save_model/try2AgentUR.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 24)                384       
_________________________________________________________________
dense_14 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 125       
Total params: 1,109
Trainable params: 1,109
Non-trainable params: 0
_________________________________________________________________


TclError: invalid command name ".!canvas"

# Save graph

In [ ]:
import pylab
import pickle

with open ('./save_data/1Agent10times10_24n_seed3', 'rb') as fp:
   s = pickle.load(fp)
with open ('./save_data/episodes', 'rb') as fp:
   episodes = pickle.load(fp)

pylab.plot(episodes, s, 'b', linewidth=0.1)

pylab.xlabel('Episodes')
pylab.ylabel('Reward')
pylab.ylim((-20, 0))
#pylab.title('Reward as function of episodes (Total steps = 27984)')

pylab.savefig("./save_graph/1Agent10times10_24n_seed3_27984step.eps", format='eps', dpi=900)

print(min(s))
print(max(s))

# Save/load lists via pickle

In [ ]:
import pickle

with open('./save_data/1Agent10times10_24n_seed3', 'wb') as fp:
    pickle.dump(scores, fp)
#with open ('./save_data/1Agent10times10', 'rb') as fp:
   #scores = pickle.load(fp)

In [ ]:
import pickle

test = agent.model.get_weights()
print(test)

with open('1Agent10times10_24n_L', 'wb') as fp:
    pickle.dump(test, fp)

In [ ]:
print(global_step)